In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from skimage.measure import regionprops, regionprops_table
import pandas as pd

import cells_in_gel.preprocess as prep
import cells_in_gel.properties as props
from cells_in_gel.properties import im_properties

In [ ]:
im_stack = io.imread('C3-NTG-CFbs_NTG5ECM_1mMRGD_20x_003.tif')
im = im_stack[10] # image 10 from the stack

label, overlay = prep.phalloidin_488_segment(im, min_size=350)

In [ ]:
x, y, z = im_stack.shape # dimensions of the image stack
label_stack = np.zeros_like(im_stack) # create empty array for label
overlay_stack = np.zeros((x, y, z, 3)) # create empty array for overlay

In [ ]:
for i in range(x):
    label_stack[i], overlay_stack[i] = prep.phalloidin_488_segment(im_stack[i], min_size=350)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 7))

axes[0].imshow(overlay_stack[1], cmap='gray')
axes[0].set_title('Original', fontsize='20')

axes[1].imshow(overlay_stack[2], cmap='gray')
axes[1].set_title('Overlay of Objects', fontsize='20')

for ax in axes:
    ax.axis('off')

In [ ]:
frames = []

#Create data frame called frames that has the image number labelled.
for i in range(len(label_stack)):
    frames.append(props.im_properties(label_stack[i], im_stack[i]))
    frames[i]['Image number'] = i

    #concatenate all the data into one big data frame
df = pd.concat(frames)

In [ ]:
def dist(a, b):
    v = ((b[0]-a[0])**2 + (b[1]-a[1])**2)**.5
    return v

# initialize cell number to an invalid value
df['cell'] = -1
# sort by image num, so can iter over whole df in order easily
df = df.sort_values('Image number', ascending = [1]).reset_index()

# figure out what cross sections are in the same cells
for j, r in df.iterrows():
    # get center of current cross section
    r_cent = [r['centroid-0'], r['centroid-1']]
    # get centers of cross sections in prev slice
    prev = df[df['Image number'] == r['Image number']-1]
    # get cross sections from prev slice that are "close" to current cross section
    # TODO fix threshold, possibly generated by code?
    close = prev[(dist(r_cent, [prev['centroid-0'], prev['centroid-1']]) < 3)]
    # if there cross section close to current, current probably part of same cell
    if not close.empty:
        # TODO come up with a smarter way to choose which cell part of, maybe closest?
        # choose an arbitrary close cell to be part of
        df['cell'].iat[j] = close['cell'].min()
    else:
        # otherwise, is a new cell, so give it a new cell number
        df['cell'].iat[j] = df['cell'].max()+1
        

df['cell'].max()